In [2]:
import os

In [3]:
%pwd

'c:\\Users\\saika\\OneDrive\\Desktop\\projects\\chicken_disease_classification\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\saika\\OneDrive\\Desktop\\projects\\chicken_disease_classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

[2024-01-23 09:06:20,274: WARNING: module_wrapper: From c:\Users\saika\OneDrive\Desktop\projects\chicken_disease_classification\chicken-env\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [19]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        try:
            config = self.config.prepare_callbacks
        except KeyError:
            # Handle the absence of 'prepare_callbacks' key, set default values, or raise an error.
            # For now, I'll set a default empty dictionary.
            config = {}

        model_ckpt_dir = os.path.dirname(config.get('checkpoint_model_filepath', ''))
        create_directories([
            Path(model_ckpt_dir),
            Path(config.get('tensorboard_root_log_dir', ''))
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.get('root_dir', '')),
            tensorboard_root_log_dir=Path(config.get('tensorboard_root_log_dir', '')),
            checkpoint_model_filepath=Path(config.get('checkpoint_model_filepath', ''))
        )

        return prepare_callback_config
    


    
    def get_training_config(self) -> TrainingConfig:
        try:
            training = self.config.training
        except KeyError:
        # Handle the absence of 'training' key, set default values, or raise an error.
        # For now, I'll set a default empty dictionary.
            training = {}

        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.get('root_dir', ''))  # Use get() to handle missing key
    ])

        training_config = TrainingConfig(
           root_dir=Path(training.get('root_dir', '')),
           trained_model_path=Path(training.get('trained_model_path', '')),
           updated_base_model_path=Path(prepare_base_model.get('updated_base_model_path', '')),
           training_data=Path(training_data),
           params_epochs=params.get('EPOCHS', 0),  # Replace 0 with a suitable default value
           params_batch_size=params.get('BATCH_SIZE', 0),  # Replace 0 with a suitable default value
           params_is_augmentation=params.get('AUGMENTATION', False),  # Replace False with a suitable default value
           params_image_size=params.get('IMAGE_SIZE', 0)  # Replace 0 with a suitable default value
    )

        return training_config



    




    
            

In [9]:
import time

In [17]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile 
import tensorflow as tf
import time

In [12]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [20]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-01-23 09:37:47,979: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-23 09:37:47,981: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-23 09:37:47,983: INFO: common: created directory at: artifacts]
[2024-01-23 09:37:47,984: INFO: common: created directory at: .]
[2024-01-23 09:37:47,985: INFO: common: created directory at: .]
[2024-01-23 09:37:47,985: INFO: common: created directory at: .]
[2024-01-23 09:37:48,232: WARNING: module_wrapper: From c:\Users\saika\OneDrive\Desktop\projects\chicken_disease_classification\chicken-env\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-01-23 09:37:48,313: WARNING: module_wrapper: From c:\Users\saika\OneDrive\Desktop\projects\chicken_disease_classification\chicken-env\Lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprec